## Spam mail分類を行う

In [ ]:
%matplotlib inline
""" span.cvsを読み込み """

import os
import sys
import csv
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('stopwords')
#from typing import List
#from gensim.corpora.dictionary import Dictionary
import numpy as np

FILE_PATH = './spam_data/spam.csv'
LABEL     = 'label'
MESSAGE   = 'message'

print(pd.__version__)

df = pd.read_csv(FILE_PATH, encoding='latin-1', header=None, names = [LABEL, MESSAGE, 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'])

print("-------------")
print("hum count   = ", sum([ x == "ham" for x in df[LABEL].values]))
print("supam count = ", sum([ x == "spam" for x in df[LABEL].values]))
print("-------------")

# ノイズを取り除く
clean_str = [x.replace('?', '') for x in df[MESSAGE]]
clean_str = [x.replace('!', '') for x in clean_str]
clean_str = [x.replace(',', '') for x in clean_str]
clean_str = [x.replace('.', '') for x in clean_str]
lower_msg = [x.lower() for x in clean_str] 

#all_token = [nltk.tokenize.word_tokenize(x) for x in lower_msg]
#print(all_token[100])
# windowsの場合には、以下のようにする
# 文書のtokenを作成する
all_tokens = [x.split() for x in lower_msg]
max_token_len = max([len(x) for x in all_tokens])
print("max_token_len = ", max_token_len)
# stop wordsを取り除く
stopwords = nltk.corpus.stopwords.words('english')
all_tokens = [[x for x in words if x not in stopwords] for words in all_tokens]
#print(all_tokens[100])
#print("keys # = ", len(all_tokens))

# all_tokensのコーパスを作成する
cop_dict = {'nishida': 0}
for token in all_tokens:
    for word in token:
        if word not in cop_dict:
            cop_dict[word] = max(cop_dict.values()) + 1            
#print("cop_dict # = ", cop_dict)

# all_tokensの中の単語を、cop_dictの単語IDに置き換える
all_token_ids = []
for token in all_tokens:
    token_ids = []
    for word in token:
        token_ids.append(cop_dict[word])
    all_token_ids.append(token_ids)
print(all_token_ids[100])

# hamとspamのラベルを数値に置き換える hum : 1 , spam : 2
id_label = []
for label in df[LABEL]:
    if label == "ham":
        id_label.append(1)
    else:
        id_label.append(2)

#print("id_label # = ", id_label[100])
#print("df[LABEL].values 100 = ", df[LABEL].values[100])




[nltk_data] Downloading package punkt to /Users/nisida/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nisida/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


2.0.2
-------------
hum count   =  4825
supam count =  747
-------------
max_token_len =  171
[665, 202, 221, 666, 667, 668, 595, 477, 21, 669, 294, 670, 553]
id_label # =  1
df[LABEL].values 100 =  ham


{'name': 1, 'price':2} ->  all_token内の全ての単語にユニークなID（数字)をふる（Pythonの辞書で作成する）. -> コーパスと呼ぶ。

気になるpoint

token1 = [1,3, 6, 7] , token2 = [10, 13, 14, 15 16, 17]


In [ ]:
import nltk 

print(nltk.__version__)